In [137]:
import numpy as np
import cv2

def harrisCorners(im, sigma):
    dim = np.ceil(sigma * 4)
    if dim % 2 != 0:
        dim+=1
    
    oneMat =  np.ones((int(dim/2), int(dim/2)), dtype=int)
    negMat = -1 * oneMat
    D_x = np.hstack((negMat, oneMat))
    D_x = np.vstack((D_x, D_x))
    D_y = np.vstack((oneMat, negMat))
    D_y = np.hstack((D_y, D_y))
    
    dx = cv2.filter2D(im, -1, D_x)
    dy = cv2.filter2D(im, -1, D_y)
    
    C_size = int(np.ceil(sigma * 5))
    if (C_size % 2 == 0):
        C_size+=1
    C_size_half = int(C_size / 2)
    threshold = np.zeros(im.shape)
    
    for i in range(im.shape[0] - C_size):
        for j in range(im.shape[1] - C_size):
            dx_kernal = dx[i:i+C_size, j:j+C_size]
            dy_kernal = dy[i:i+C_size, j:j+C_size]
            dx2 = np.sum(np.square(dx_kernal))
            dy2 = np.sum(np.square(dy_kernal))
            dxdy = np.sum(np.multiply(dx_kernal, dy_kernal))
            C_det = np.subtract(np.multiply(dx2, dy2), np.square(dxdy))
            C_tr = np.add(dx2, dy2)
            threshold[i + C_size_half, j + C_size_half] = C_det - 0.05 * np.square(C_tr)

    thresh_size = 29
    thresh_size_half = int(thresh_size / 2) 
    thresholdAvg = np.mean(abs(threshold))
    corners=[]
    
    for i in range(im.shape[0]-thresh_size):
        for j in range(im.shape[1]-thresh_size):
            if threshold[i+thresh_size_half,j+thresh_size_half] > 0 and abs(threshold[i+thresh_size_half,j+thresh_size_half]) > thresholdAvg:
                thresh = threshold[i:i+thresh_size,j:j+thresh_size]
                if threshold[i+thresh_size_half,j+thresh_size_half] == np.max(thresh):
                    corners.append([i+thresh_size_half,j+thresh_size_half])
    return np.array(corners)

def SSD(im1, im2, cnr1, cnr2):
    SSD = np.zeros((len(cnr1), len(cnr2)))
    for i in range(len(cnr1)):
        for j in range(len(cnr2)):
            f1 = im1[cnr1[i, 0] - 12 : cnr1[i, 0] + 13, cnr1[i, 1] - 12 : cnr1[i, 1] + 13]
            f2 = im2[cnr2[j, 0] - 12 : cnr2[j, 0] + 13, cnr2[j, 1] - 12 : cnr2[j, 1] + 13]
            SSD[i, j] = np.sum(np.square(np.subtract(f1, f2)))
            
    thresh = np.mean(SSD) * 4.5
    pts = []
    for i in range(len(cnr1)):
        for j in range(len(cnr2)):
            if SSD[i, j] == np.min(SSD[i,:]) and SSD[i,j] < thresh:
                min_local_value = SSD[i,j]
                SSD[i,j] = np.max(SSD[i,:])
            else:
                continue
            if min_local_value / np.min(SSD[i,:]) < 0.7:
                SSD[:,j] = np.max(SSD)
                SSD[i,j] = min_local_value
                pts.append([cnr1[i],cnr2[j]])
    return np.array(pts)

def NCC(im1, im2, cnr1, cnr2):
    NCC = np.zeros((len(cnr1), len(cnr2)))   
    for i in range(len(cnr1)):
        for j in range(len(cnr2)):
            f1 = im1[cnr1[i, 0] - 12 : cnr1[i, 0] + 13, cnr1[i, 1] - 12 : cnr1[i, 1] + 13]
            f2 = im2[cnr2[j, 0] - 12 : cnr2[j, 0] + 13, cnr2[j, 1] - 12 : cnr2[j, 1] + 13]
            ncc = np.sum(np.multiply(np.subtract(f1, np.mean(f1)), np.subtract(f2, np.mean(f2))))
            ncc /= np.sqrt(np.sum(np.square(np.subtract(f1, np.mean(f1)))) * np.sum(np.square(np.subtract(f2, np.mean(f2)))))
            NCC[i, j] = ncc

    pts=[]
    for i in range(len(cnr1)):
        for j in range(len(cnr2)):
            if NCC[i,j]==np.max(NCC[i,:]) and NCC[i,j]>0.8:
                pts.append([[cnr1[i,0],cnr1[i,1]],[cnr2[j,0],cnr2[j,1]]])

    return np.array(pts)

def showPoints(im1, im2, corners):    
    im = np.zeros((max(im1.shape[0], im2.shape[0]), im1.shape[1] + im2.shape[1],3))
    im[0 : im1.shape[0], 0 : im1.shape[1]] = im1
    im[0 : im2.shape[0], im1.shape[1] : im1.shape[1] + im2.shape[1]] = im2
    
    for cnr in corners:
        cv2.circle(im,(cnr[0, 1], cnr[0, 0]),2,(255,0,0),1)
        cv2.circle(im,(im1.shape[1]+cnr[1, 1],cnr[1, 0]),2,(255,0,0),1)
        cv2.line(im,(cnr[0, 1],cnr[0, 0]),(im1.shape[1]+cnr[1, 1],cnr[1, 0]), (255,0,0))
    return im

def processHw(sigma, im1Path, im2Path, out1, out2):
    im1 = cv2.imread(im1Path)
    im2 = cv2.imread(im2Path)

    im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)

    im1Corners = harrisCorners(im1Gray, sigma)
    im2Corners = harrisCorners(im2Gray, sigma)

    ssdPts = SSD(im1Gray,im2Gray,im1Corners,im2Corners)
    imSSD = showPoints(im1, im2, ssdPts)
    cv2.imwrite(out1, imSSD)

    nccPts = NCC(im1Gray, im2Gray, im1Corners, im2Corners)
    imNCC = showPoints(im1, im2, nccPts)
    cv2.imwrite(out2, imNCC)    
    
#processHw(0.7, 'Images/pair1/1.JPG', 'Images/pair1/2.JPG', 'Results/Pair1SSD7.jpg', 'Results/Pair1NCC7.jpg')
#processHw(1.0, 'Images/pair1/1.JPG', 'Images/pair1/2.JPG', 'Results/Pair1SSD1.jpg', 'Results/Pair1NCC1.jpg')
#processHw(1.2, 'Images/pair1/1.JPG', 'Images/pair1/2.JPG', 'Results/Pair1SSD12.jpg', 'Results/Pair1NCC12.jpg')
#processHw(2.0, 'Images/pair1/1.JPG', 'Images/pair1/2.JPG', 'Results/Pair1SSD2.jpg', 'Results/Pair1NCC2.jpg')

#processHw(0.7, 'Images/pair2/1.JPG', 'Images/pair2/2.JPG', 'Results/Pair2SSD7.jpg', 'Results/Pair2NCC7.jpg')
#processHw(1.0, 'Images/pair2/1.JPG', 'Images/pair2/2.JPG', 'Results/Pair2SSD1.jpg', 'Results/Pair2NCC1.jpg')
#processHw(1.2, 'Images/pair2/1.JPG', 'Images/pair2/2.JPG', 'Results/Pair2SSD12.jpg', 'Results/Pair2NCC12.jpg')
#processHw(2.0, 'Images/pair2/1.JPG', 'Images/pair2/2.JPG', 'Results/Pair2SSD2.jpg', 'Results/Pair2NCC2.jpg')

#processHw(0.7, 'Images/pair3/1.jpg', 'Images/pair3/2.jpg', 'Results/Pair3SSD7.jpg', 'Results/Pair3NCC7.jpg')
#processHw(1.0, 'Images/pair3/1.jpg', 'Images/pair3/2.jpg', 'Results/Pair3SSD1.jpg', 'Results/Pair3NCC1.jpg')
#processHw(1.2, 'Images/pair3/1.jpg', 'Images/pair3/2.jpg', 'Results/Pair3SSD12.jpg', 'Results/Pair3NCC12.jpg')
#processHw(2.0, 'Images/pair3/1.jpg', 'Images/pair3/2.jpg', 'Results/Pair3SSD2.jpg', 'Results/Pair3NCC2.jpg')

#processHw(0.7, 'Images/pair4/1.JPG', 'Images/pair4/2.JPG', 'Results/Pair4SSD7.jpg', 'Results/Pair4NCC7.jpg')
#processHw(1.0, 'Images/pair4/1.JPG', 'Images/pair4/2.JPG', 'Results/Pair4SSD1.jpg', 'Results/Pair4NCC1.jpg')
#processHw(1.2, 'Images/pair4/1.JPG', 'Images/pair4/2.JPG', 'Results/Pair4SSD12.jpg', 'Results/Pair4NCC12.jpg')
#processHw(2.0, 'Images/pair4/1.JPG', 'Images/pair4/2.JPG', 'Results/Pair4SSD2.jpg', 'Results/Pair4NCC2.jpg')

#processHw(0.7, 'Images/pair4/1.JPG', 'Images/pair4/2.JPG', 'Results/Pair4SSD7.jpg', 'Results/Pair4NCC7.jpg')
#processHw(1.0, 'Images/pair4/1.JPG', 'Images/pair4/2.JPG', 'Results/Pair4SSD1.jpg', 'Results/Pair4NCC1.jpg')
#processHw(1.2, 'Images/pair4/1.JPG', 'Images/pair4/2.JPG', 'Results/Pair4SSD12.jpg', 'Results/Pair4NCC12.jpg')
#processHw(2.0, 'Images/pair4/1.JPG', 'Images/pair4/2.JPG', 'Results/Pair4SSD2.jpg', 'Results/Pair4NCC2.jpg')

#processHw(0.7, 'Images/pair5/1.jpg', 'Images/pair5/2.jpg', 'Results/Pair5SSD7.jpg', 'Results/Pair5NCC7.jpg')
#processHw(1.0, 'Images/pair5/1.jpg', 'Images/pair5/2.jpg', 'Results/Pair5SSD1.jpg', 'Results/Pair5NCC1.jpg')
#processHw(1.2, 'Images/pair5/1.jpg', 'Images/pair5/2.jpg', 'Results/Pair5SSD12.jpg', 'Results/Pair5NCC12.jpg')
processHw(2.0, 'Images/pair5/1.jpg', 'Images/pair5/2.jpg', 'Results/Pair5SSD2.jpg', 'Results/Pair5NCC2.jpg')

SyntaxError: invalid syntax (<ipython-input-137-2f31860e2051>, line 119)

In [141]:
import cv2
cv2.__version__

'4.0.1'